# QSAR modelling and Representation Building

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error
from sentence_transformers import SentenceTransformer

import logging
logging.basicConfig(format="%(message)s")
log = logging.getLogger(__name__)
log.setLevel(logging.INFO)

In [ ]:
log.info(f"RDKit version {rdkit.__version__} Pandas version: {pd.__version__} Numpy version {np.__version__}") 

## Load the data set

In [ ]:
data = pd.read_csv("/Users/James/Documents/Carbon-capture-fingerprint-generation/ccs-datasets/ccs-98.csv")
data.columns = [ent.strip() for ent in data.columns]

In [ ]:
data.shape

In [ ]:
data.head()

Canonicalize the smiles

In [ ]:
data["smiles"] = [Chem.MolToSmiles(Chem.MolFromSmiles(smi), canonical=True) for smi in data["smiles"].values]

In [ ]:
data

Genertate molecules from the inchi

In [ ]:
data.drop_duplicates(subset="smiles", keep="first", inplace=True)

In [ ]:
data

In [ ]:
data["rdkit_molecule"] = [Chem.MolFromSmiles(s) for s in data["smiles"].values]

## Molecular descriptors

### Using RDKit

In [ ]:
descriptors = pd.DataFrame()
for rep, m in zip(data["smiles"].values, data["rdkit_molecule"].values):
    tmp_df = pd.DataFrame(Chem.Descriptors.CalcMolDescriptors(m, np.nan), index=[rep])
    descriptors = pd.concat([descriptors, tmp_df])

### Fingerprints

In [ ]:
morgan = np.array([Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, useChirality=True, radius=2, nBits=1024) for mol in tqdm(data["rdkit_molecule"])])

In [ ]:
maccs = np.array([Chem.rdMolDescriptors.GetMACCSKeysFingerprint(mol) for mol in tqdm(data["rdkit_molecule"])])

### Embeddings

In [ ]:
model_st = SentenceTransformer("jonghyunlee/MoleculeBERT_ChEMBL-pretrained")
embeddings = model_st.encode(data["smiles"].values[:100])
log.info(f"The embedding is length {len(embeddings)}{os.linesep}{embeddings}")

## QSAR

In [ ]:
data.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(maccs,
                                                   data["capacity_molco2_molamime"],
                                                   test_size=0.10, 
                                                   random_state=15715, 
                                                   shuffle=True)

log.info(f"There are {len(y_train)} training points and {len(y_test)} data points for the QSAR model")

dnn_X_train, dnn_X_validate, dnn_y_train, dnn_y_validate = train_test_split(X_train,
                                                                           y_train,
                                                                           test_size=0.10, 
                                                                           random_state=15715, 
                                                                           shuffle=True)

In [ ]:
def build_model(reg, param_grid: dict, features:list, classes:list, cv:int = 5, 
                scorers: list = ["r2", "neg_mean_squared_error"], 
                refit:str = "mean_squared_error"):
    """
    Function to build a regressor
    :param reg: regressor - sklearn instantiation of a regressor
    :param param_grid: dict - dictionary of arguments for the regressor
    :param cv: int - number of cross validation windows
    :param scorers: list - list of scoring functions
    :param refit: scorer to use to refit to get the best model parameters
    :return regressor with the best parameters
    """
    
    gs = GridSearchCV(reg, 
                  param_grid,
                  cv=cv,
                  scoring=scorers,
                  refit=refit,
                  verbose=1 #3,
                  ).fit(pd.DataFrame(features), classes)

    log.info(f"Best parameters are: {gs.best_params_}")

    reg= gs.best_estimator_
    
    return reg

regressor = RandomForestRegressor(random_state=15715)
scorers = ["r2", "neg_mean_absolute_percentage_error", "neg_root_mean_squared_error"]
param_grid = {"max_depth": [3, 5, 10],
              "min_samples_split": [2, 5, 7, 10],
              "n_estimators": [5, 10, 50, 100, 200],
              "max_features": ["sqrt", "log2"]
}

regressor = build_model(regressor, param_grid, X_train, y_train,
                  scorers=scorers, refit="neg_root_mean_squared_error")

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
pre = regressor.predict(X_test)
mae = mean_absolute_error(y_test, pre)
mape = mean_absolute_percentage_error(y_test, pre)
mse = mean_squared_error(y_test, pre)
cod = r2_score(y_test, pre)

log.info(f"MAE: {mae:.4f} MAPE: {mape:.4f} MSE: {mse:.4f} RMSE: {np.sqrt(mse):.4f} CoD (r2): {cod:.4f}")